In [3]:
import pandas as pd
import numpy as np

## My F-beta score

Difference with default F1 score:
When deal with tricky metric like this
<pre>
|         |      |     True    |
|         |      |   0  |   1  |
|---------|------|------|------|
| Predict |   0  |   3  |   0  |
|         |   1  |   4  |   0  |
</pre>
No real positive samples makes the denominator and numeratore of F1 metric equal to 0. F1, recall, precision = 0/0.
Default F1 test are all equal to 0. But in my opinion: F1, recall, precision score are 0, 1, 0 respectively.

-------
And this:
          True
           0
Predict   0   3
In this case, I will set all of F1, recall, precision to 1. However, the default is 0.

--------
Besides, my version could return the cross table as above.

--------

Note about Beta:
![title](f1score.svg)
beta < 1 lends more weight to precision, while beta > 1 favors recall

In [4]:
def Score (PredictList,TrueList,beta):
    X=pd.concat([pd.DataFrame(PredictList),pd.DataFrame(TrueList)],axis=1, ignore_index=True)
    X['count']=1
    X.columns=['Predict','True','count']
    Acc=X.pivot_table(index='Predict',columns='True',values='count',aggfunc=np.sum)
    Acc=Acc.fillna(value=0)
    rate,f1,recall,precision=0,0,0,0
    b=beta**2
    if Acc.shape==(2,2):
        rate=rate+(float(Acc[0][0]+Acc[1][1])/Acc.sum().sum())
        if Acc[1][1]!=0:
            recall=float(Acc[1][1])/float(Acc[1][0]+Acc[1][1])
            precision=float(Acc[1][1])/float(Acc[0][1]+Acc[1][1])
            f1=(1+b) * (float(Acc[1][1])/float(Acc[0][1]+Acc[1][1])) * (float(Acc[1][1])/float(Acc[1][0]+Acc[1][1])) / ((b * float(Acc[1][1])/float(Acc[0][1]+Acc[1][1])) + (float(Acc[1][1])/float(Acc[1][0]+Acc[1][1])))
    elif Acc.shape==(1,2):
        if Acc.index[0]==0:
            rate=rate+(float(Acc[0][0])/Acc.sum().sum())
        else:
            rate=float(Acc[0][1])/Acc.sum().sum()
            recall=1
            precision=float(Acc[0][1])/Acc.sum().sum()
            f1=(1+b) * ((float(Acc[0][1])/Acc.sum().sum()) * 1) / ((b * float(Acc[0][1])/Acc.sum().sum()) + 1)
    elif Acc.shape==(2,1):
        if Acc.columns==1:
            rate=float(Acc.iloc[1])/Acc.sum().sum()
            recall=float(Acc.iloc[1])/Acc.sum().sum()
            precision=1
            f1=(1+b) * float(Acc.iloc[1])/Acc.sum().sum()/(float(Acc.iloc[1])/Acc.sum().sum()+ b)
        else:
            rate=float(Acc.iloc[0])/Acc.sum().sum()
            recall=1
    else: #Acc.shape=(1,1)
        rate=1
        recall=1
        precision=1
        f1=1
    return Acc,rate,f1,recall,precision

# Import Y

In [236]:
Y=pd.read_csv('comm-at-least-4-Y.csv',header=0,index_col=0)
Y=Y.sort_index(axis=0, level=None, ascending=True)
Y=Y.fillna(value=0)

In [277]:
Y_political=Y.sum(axis=1)
Y_political[Y_political>0]=1

In [254]:
print Y_political.sum()

1076


In [289]:
Y_political_yes=Y[Y_political>0]
metric_variance_political_yes=metric_variance[np.where(Y_political>0)]

In [281]:
Y_political_yes=Y[classifiers["MulBayes"]["f"].predict(metric_variance)>0]
metric_variance_political_yes=metric_variance[np.where(classifiers["MulBayes"]["f"].predict(metric_variance)>0)]

In [282]:
print Y_political_yes.sum()

Yellow    722
Blue       19
Local      86
dtype: int64



# Import and Transform X

In [171]:
likeall=pd.read_csv("./comm-at-least-4-engage.txt",header=None, names=['time','pageid','uid'],sep=' ')
likeall['count']=1

In [172]:
new_metric=likeall.pivot_table(index='uid',columns='pageid',values='count',aggfunc=np.sum)

In [27]:
#Save metric
new_metric.to_csv(path_or_buf='./comm-at-least-4-engage-metric.csv')

In [174]:
new_metric=new_metric.fillna(value=0)

## Feature Selection (1): Variance

In [175]:
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(threshold=0.09)
metric_variance=selector.fit_transform(new_metric)

In [81]:
print new_metric.shape
print metric_variance.shape

(2815, 970)
(2815L, 61L)


# Split Test Set

In [290]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(metric_variance_political_yes, Y_political_yes['Local'], test_size=0.0)
y_train=np.asarray(y_train,dtype="float64")
y_test=np.asarray(y_test,dtype="float64")

In [29]:
print X_test.shape, y_test.shape,X_train.shape,y_train.shape

(282L, 61L) (282L,) (2533L, 61L) (2533L,)


In [286]:
y_train.sum()

727.0

In [12]:
X_selected.shape

(1515L, 67L)

## Under-sampling

In [181]:
list=np.where(y_train==0)
len(list[0])

1217

In [182]:
del_index=np.random.choice(list[0],size=len(list[0])/2, replace=False)

In [183]:
y_train=np.delete(y_train,del_index, None)
X_train=np.delete(X_train,del_index, 0)

In [184]:
print y_train.shape
print X_train.shape

(709L,)
(709L, 61L)


## Over-sampling SMOTE

In [230]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(kind='regular',ratio=0.3)
X_train, y_train = sm.fit_sample(X_train, y_train)

In [284]:
print X_train.shape
print len(y_train[y_train==1])

(3151L, 61L)
727


# PCA

In [101]:
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
#new_metric=new_metric.fillna(value=0)
metric_pca=pca.fit_transform(X_train)
#metric_pca=pca.fit_transform(pre_metric)

In [146]:
pca.explained_variance_ratio_.sum()

0.58833931652814542

In [862]:
print metric_pca.shape

(1015L, 10L)


# NMF

In [874]:
from sklearn.decomposition import NMF
model = NMF(n_components=10, init='random',alpha=0.01,max_iter=500,tol=1e-4,random_state=0)
model=model.fit(X_train)
metric_nmf=model.transform(X_train)

In [875]:
X_test=model.transform(X_test)

In [35]:
#component-page
model.components_.shape
#user-component
metric_nmf.shape

(1815L, 10L)

###  Assign X_train

In [168]:
X_train=new_metric

In [113]:
X_train=metric_variance

In [97]:
X_train=metric_pca

In [867]:
X_train=metric_nmf

In [868]:
X_train.shape

(1015L, 10L)

## Feature Selection (2) : Coefficient p value

In [37]:
from scipy import stats
from sklearn.linear_model import LogisticRegression
LogReg=LogisticRegression()
LogReg.fit(X_selected, Y_label)
sse = np.sum((LogReg.predict(X_selected) - Y_label) ** 2, axis=0) / float(X_selected.shape[0] - X_selected.shape[1])
se = np.array([
        np.sqrt(np.diagonal(sse * np.linalg.inv(np.dot(X_selected.T, X_selected))))
    ])
t = LogReg.coef_ / se
p = 2 * (1 - stats.t.cdf(np.abs(t), Y_label.shape[0] - X_selected.shape[1]))
np.set_printoptions(suppress=True)
print 'How many variables are significantly related to Y:',len(p[p<0.05])
print p

How many variables are significantly related to Y: 9
[[ 0.          0.          0.          0.08084395  0.00173334  0.          0.
   0.          0.00021892  0.        ]]


In [38]:
from sklearn.feature_selection import RFE
selector = RFE(LogReg, 9, step=1)
selector = selector.fit(X_selected, Y_label)
selector.support_

array([ True,  True,  True, False,  True,  True,  True,  True,  True,  True], dtype=bool)

In [39]:
X_selected=X_selected[:,selector.support_]
X_selected.shape

(1815L, 9L)

## Feature Selection (3): Chi2

In [148]:
# Chi Square
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [154]:
chi2(X_train, y_train)[1].argmax()

16

In [314]:
#p value
len(chi2(X_train, y_train)[1][chi2(X_train, y_train)[1]<0.05])

56

In [315]:
SKB = SelectKBest(chi2, k=56).fit(X_train, y_train)

In [316]:
X_train=SKB.transform(X_train)
X_test=SKB.transform(X_test)

# Model Comparison

In [291]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVR

#cross-validation
from sklearn.cross_validation import KFold
k=10
kf = KFold(len(y_train), n_folds=k,shuffle=True)

classifiers = {"MulBayes": {"f": MultinomialNB()},
               #"RandomForest": {"f": RandomForestClassifier()},
               #"DecisionTree": {"f": DecisionTreeClassifier()},
               #"LogReg": {"f": LogisticRegression()},
               #"SVR": {"f":SVR(kernel='rbf', gamma=0.1)}
              }

beta=1
for model,color in zip(classifiers.keys(),colors):
    recall,rate,precision,f1 = 0.0,0.0,0.0,0.0
    n=0
    for train_index, test_index in kf:
        # Fit
        classifiers[model]["f"].fit(X_train[train_index], y_train[train_index])
        # Predict
        if model=="SVR":
            predicts=classifiers[model]["f"].predict(X_train[test_index])
            predicts[predicts<=0.5]=0
            predicts[predicts>0.5]=1
            Acc,rate_fold,f1_fold,recall_fold,precision_fold=Score(predicts,y_train[test_index],beta)
        else:
            Acc,rate_fold,f1_fold,recall_fold,precision_fold=Score(classifiers[model]["f"].predict(X_train[test_index]),y_train[test_index],beta)
        rate+=rate_fold
        f1+=f1_fold
        recall+=recall_fold
        precision+=precision_fold
    rate=float(rate)/k
    recall=float(recall)/k
    precision=float(precision)/k
    f1=float(f1)/k
    classifiers[model]["Scores"]={"accuracy":rate, "f-beta":f1, "recall":recall, "precision":precision}
    classifiers[model]["predict"] = classifiers[model]["f"].predict(X_train)
    if model!="SVR":
        classifiers[model]["predict_prob"] = classifiers[model]["f"].predict_proba(X_train)
    else:
        classifiers[model]["predict_prob"] = np.asarray([[1-i,i] for i in classifiers[model]["f"].predict(X_train)])

# Evaluate the performance
results = pd.DataFrame(columns=classifiers[classifiers.keys()[0]]["Scores"].keys())

for model in classifiers.keys():
    results.loc[model] = classifiers[model]["Scores"].values()

print results[["accuracy","f-beta","recall","precision"]]

          accuracy    f-beta    recall  precision
MulBayes  0.868917  0.185026  0.155255   0.251753


# Test

In [251]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

colors = [  'yellow','seagreen','cyan', 'indigo', 'blue', 'darkorange']
lw = 2

results = pd.DataFrame(columns=["accuracy","f-beta","recall","precision"])

for model,color in zip(classifiers.keys(),colors):
    if model=="SVR":
        predicts=classifiers[model]["f"].predict(X_test)
        predicts[predicts<=0.5]=0
        predicts[predicts>0.5]=1
        Acc,rate,f1,recall,precision=Score(predicts,y_test,beta)
        fpr, tpr, thresholds = roc_curve(y_test, classifiers[model]["f"].predict(X_test))
    else:
        Acc,rate,f1,recall,precision=Score(classifiers[model]["f"].predict(X_test),y_test,beta)
        fpr, tpr, thresholds = roc_curve(y_test, classifiers[model]["f"].predict_proba(X_test)[:,-1])
    results.loc[model]=rate,f1,recall,precision
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=lw, color=color,label=model+'(area = %0.2f)' % (roc_auc))
plt.plot([0, 1], [0, 1], linestyle='--', lw=lw, color='k',label='Random')
print results
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.title('ROC curve - Yellow')
plt.show()


          accuracy    f-beta    recall  precision
MulBayes  0.723404  0.632075  0.663366   0.603604
